# Vexpresso Walkthrough

## In this walkthrough, we will be using a small dataset of Pokemon statistics to showcase vexpresso's capabilities

### Imports

In [1]:
import json
import pandas as pd
from langchain.embeddings import HuggingFaceHubEmbeddings, HuggingFaceEmbeddings
import numpy as np
import vexpresso
from vexpresso.retriever import NumpyRetriever
from vexpresso import transformation

/home/shyam/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Lets look at the Data: from https://github.com/Purukitto/pokemon-data.json

In [2]:
with open("./data/pokedex.json", 'r') as f:
    stuff = json.load(f)

df = pd.DataFrame(stuff)

In [3]:
df.head(10)

,id,name,type,base,species,description,evolution,profile,image
0,1,"{'english': 'Bulbasaur', 'japanese': 'フシギダネ', ...","[Grass, Poison]","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. A...",Seed Pokémon,Bulbasaur can be seen napping in bright sunlig...,"{'next': [['2', 'Level 16']]}","{'height': '0.7 m', 'weight': '6.9 kg', 'egg':...",{'sprite': 'https://raw.githubusercontent.com/...
1,2,"{'english': 'Ivysaur', 'japanese': 'フシギソウ', 'c...","[Grass, Poison]","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. A...",Seed Pokémon,There is a bud on this Pokémon’s back. To supp...,"{'prev': ['1', 'Level 16'], 'next': [['3', 'Le...","{'height': '1 m', 'weight': '13 kg', 'egg': ['...",{'sprite': 'https://raw.githubusercontent.com/...
2,3,"{'english': 'Venusaur', 'japanese': 'フシギバナ', '...","[Grass, Poison]","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. A...",Seed Pokémon,There is a large flower on Venusaur’s back. Th...,"{'prev': ['2', 'Level 32']}","{'height': '2 m', 'weight': '100 kg', 'egg': [...",{'sprite': 'https://raw.githubusercontent.com/...
3,4,"{'english': 'Charmander', 'japanese': 'ヒトカゲ', ...",[Fire],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. A...",Lizard Pokémon,The flame that burns at the tip of its tail is...,"{'next': [['5', 'Level 16']]}","{'height': '0.6 m', 'weight': '8.5 kg', 'egg':...",{'sprite': 'https://raw.githubusercontent.com/...
4,5,"{'english': 'Charmeleon', 'japanese': 'リザード', ...",[Fire],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. A...",Flame Pokémon,Charmeleon mercilessly destroys its foes using...,"{'prev': ['4', 'Level 16'], 'next': [['6', 'Le...","{'height': '1.1 m', 'weight': '19 kg', 'egg': ...",{'sprite': 'https://raw.githubusercontent.com/...
5,6,"{'english': 'Charizard', 'japanese': 'リザードン', ...","[Fire, Flying]","{'HP': 78, 'Attack': 84, 'Defense': 78, 'Sp. A...",Flame Pokémon,Charizard flies around the sky in search of po...,"{'prev': ['5', 'Level 36']}","{'height': '1.7 m', 'weight': '90.5 kg', 'egg'...",{'sprite': 'https://raw.githubusercontent.com/...
6,7,"{'english': 'Squirtle', 'japanese': 'ゼニガメ', 'c...",[Water],"{'HP': 44, 'Attack': 48, 'Defense': 65, 'Sp. A...",Tiny Turtle Pokémon,Squirtle’s shell is not merely used for protec...,"{'next': [['8', 'Level 16']]}","{'height': '0.5 m', 'weight': '9 kg', 'egg': [...",{'sprite': 'https://raw.githubusercontent.com/...
7,8,"{'english': 'Wartortle', 'japanese': 'カメール', '...",[Water],"{'HP': 59, 'Attack': 63, 'Defense': 80, 'Sp. A...",Turtle Pokémon,"Its tail is large and covered with a rich, thi...","{'prev': ['7', 'Level 16'], 'next': [['9', 'Le...","{'height': '1 m', 'weight': '22.5 kg', 'egg': ...",{'sprite': 'https://raw.githubusercontent.com/...
8,9,"{'english': 'Blastoise', 'japanese': 'カメックス', ...",[Water],"{'HP': 79, 'Attack': 83, 'Defense': 100, 'Sp. ...",Shellfish Pokémon,Blastoise has water spouts that protrude from ...,"{'prev': ['8', 'Level 36']}","{'height': '1.6 m', 'weight': '85.5 kg', 'egg'...",{'sprite': 'https://raw.githubusercontent.com/...
9,10,"{'english': 'Caterpie', 'japanese': 'キャタピー', '...",[Bug],"{'HP': 45, 'Attack': 30, 'Defense': 35, 'Sp. A...",Worm Pokémon,"Its body is soft and weak. In nature, its perp...","{'next': [['11', 'Level 7']]}","{'height': '0.3 m', 'weight': '2.9 kg', 'egg':...",{'sprite': 'https://raw.githubusercontent.com/...


## Create Collection

In [4]:
collection = vexpresso.create(
    data="./data/pokedex.json",
)

2023-06-06 17:26:55.073 | INFO     | daft.context:runner:88 - Using PyRunner


In [5]:
collection.show(5)

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,descriptionUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64
1,"{'chinese': '妙蛙种子', 'english': 'Bulbasaur', 'french': 'Bu...","['Grass', 'Poison']","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. Attack': 65,...",Seed Pokémon,Bulbasaur can be seen napping in bright sunlight. There i...,"{'next': [['2', 'Level 16']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,0
2,"{'chinese': '妙蛙草', 'english': 'Ivysaur', 'french': 'Herbi...","['Grass', 'Poison']","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. Attack': 80,...",Seed Pokémon,There is a bud on this Pokémon’s back. To support its wei...,"{'next': [['3', 'Level 32']], 'prev': ['1', 'Level 16']}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,1
3,"{'chinese': '妙蛙花', 'english': 'Venusaur', 'french': 'Flor...","['Grass', 'Poison']","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. Attack': 100...",Seed Pokémon,There is a large flower on Venusaur’s back. The flower is...,"{'next': None, 'prev': ['2', 'Level 32']}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,2
4,"{'chinese': '小火龙', 'english': 'Charmander', 'french': 'Sa...",['Fire'],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. Attack': 60,...",Lizard Pokémon,The flame that burns at the tip of its tail is an indicat...,"{'next': [['5', 'Level 16']], 'prev': None}","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,3
5,"{'chinese': '火恐龙', 'english': 'Charmeleon', 'french': 'Re...",['Fire'],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. Attack': 80,...",Flame Pokémon,Charmeleon mercilessly destroys its foes using its sharp ...,"{'next': [['6', 'Level 36']], 'prev': ['4', 'Level 16']}","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,4


In [6]:
hf_embeddings = HuggingFaceEmbeddings()
@transformation
def hf_embedding_function(content):
    return np.array(hf_embeddings.embed_documents(content))

In [7]:
collection = collection.embed(content=list(df["description"]), column_name="description", embedding_fn=hf_embedding_function).execute()

In [8]:
collection.show(5)

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64,descriptionUtf8,embeddings_descriptionPython
1,"{'chinese': '妙蛙种子', 'english': 'Bulbasaur', 'french': 'Bu...","['Grass', 'Poison']","{'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. Attack': 65,...",Seed Pokémon,"{'next': [['2', 'Level 16']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,0,Bulbasaur can be seen napping in bright sunlight. There i...,"<np.ndarrayshape=(768,)dtype=float64>"
2,"{'chinese': '妙蛙草', 'english': 'Ivysaur', 'french': 'Herbi...","['Grass', 'Poison']","{'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. Attack': 80,...",Seed Pokémon,"{'next': [['3', 'Level 32']], 'prev': ['1', 'Level 16']}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,1,There is a bud on this Pokémon’s back. To support its wei...,"<np.ndarrayshape=(768,)dtype=float64>"
3,"{'chinese': '妙蛙花', 'english': 'Venusaur', 'french': 'Flor...","['Grass', 'Poison']","{'HP': 80, 'Attack': 82, 'Defense': 83, 'Sp. Attack': 100...",Seed Pokémon,"{'next': None, 'prev': ['2', 'Level 32']}","{'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,2,There is a large flower on Venusaur’s back. The flower is...,"<np.ndarrayshape=(768,)dtype=float64>"
4,"{'chinese': '小火龙', 'english': 'Charmander', 'french': 'Sa...",['Fire'],"{'HP': 39, 'Attack': 52, 'Defense': 43, 'Sp. Attack': 60,...",Lizard Pokémon,"{'next': [['5', 'Level 16']], 'prev': None}","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,3,The flame that burns at the tip of its tail is an indicat...,"<np.ndarrayshape=(768,)dtype=float64>"
5,"{'chinese': '火恐龙', 'english': 'Charmeleon', 'french': 'Re...",['Fire'],"{'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. Attack': 80,...",Flame Pokémon,"{'next': [['6', 'Level 36']], 'prev': ['4', 'Level 16']}","{'ability': [['Blaze', 'false'], ['Solar Power', 'true']]...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,4,Charmeleon mercilessly destroys its foes using its sharp ...,"<np.ndarrayshape=(768,)dtype=float64>"


## Querying the collection's embeddings, lazy execution
### This doesn't run anything until `execute()` is called

In [9]:
sleepy_pokemon = collection.query("embeddings_description", query="Loves to sleep", k=10)
sleepy_pokemon.df

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64,descriptionUtf8,embeddings_descriptionPython,embeddings_description_scoreFloat64


In [13]:
sleepy_pokemon.execute().show(5)

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64,descriptionUtf8,embeddings_descriptionPython,embeddings_description_scoreFloat64
143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'french': 'Ronfl...",['Normal'],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp. Attack': 6...",Sleeping Pokémon,"{'next': None, 'prev': ['446', 'high Friendship']}","{'ability': [['Immunity', 'false'], ['Thick Fat', 'false'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,142,It has no interest in anything other than eating. Even if...,"<np.ndarrayshape=(768,)dtype=float64>",0.561686
25,"{'chinese': '皮卡丘', 'english': 'Pikachu', 'french': 'Pikac...",['Electric'],"{'HP': 35, 'Attack': 55, 'Defense': 40, 'Sp. Attack': 50,...",Mouse Pokémon,"{'next': [['26', 'use Thunder Stone']], 'prev': ['172', '...","{'ability': [['Static', 'false'], ['Lightning Rod', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,24,"While sleeping, it generates electricity in the sacs in i...","<np.ndarrayshape=(768,)dtype=float64>",0.474437
328,"{'chinese': '大颚蚁', 'english': 'Trapinch', 'french': 'Krak...",['Ground'],"{'HP': 45, 'Attack': 100, 'Defense': 45, 'Sp. Attack': 45...",Ant Pit Pokémon,"{'next': [['329', 'Level 35']], 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Arena Trap', 'f...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,327,"It builds nests that double as traps, lying in wait for i...","<np.ndarrayshape=(768,)dtype=float64>",0.450725
722,"{'chinese': '木木枭', 'english': 'Rowlet', 'french': 'Brindi...","['Grass', 'Flying']","{'HP': 68, 'Attack': 55, 'Defense': 55, 'Sp. Attack': 50,...",Grass Quill Pokémon,"{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Reach', 'true'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,721,"It feels relaxed in tight, dark places and has been known...","<np.ndarrayshape=(768,)dtype=float64>",0.445208
554,"{'chinese': '火红不倒翁', 'english': 'Darumaka', 'french': 'Da...",['Fire'],"{'HP': 70, 'Attack': 90, 'Defense': 45, 'Sp. Attack': 15,...",Zen Charm Pokémon,"{'next': [['555', 'use Ice Stone']], 'prev': None}","{'ability': [['Hustle', 'false'], ['Inner Focus', 'true']...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,553,"When it sleeps, it pulls its limbs into its body and its ...","<np.ndarrayshape=(768,)dtype=float64>",0.430829


#### You can also pass in query embeddings directly

In [14]:
embedding = hf_embedding_function.func(["Loves to sleep"])
sleepy_pokemon = collection.query("embeddings_description", query_embeddings=embedding, k=10)
sleepy_pokemon.show(5)

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64,descriptionUtf8,embeddings_descriptionPython,embeddings_description_scoreFloat64
143,"{'chinese': '卡比兽', 'english': 'Snorlax', 'french': 'Ronfl...",['Normal'],"{'HP': 160, 'Attack': 110, 'Defense': 65, 'Sp. Attack': 6...",Sleeping Pokémon,"{'next': None, 'prev': ['446', 'high Friendship']}","{'ability': [['Immunity', 'false'], ['Thick Fat', 'false'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,142,It has no interest in anything other than eating. Even if...,"<np.ndarrayshape=(768,)dtype=float64>",0.561686
25,"{'chinese': '皮卡丘', 'english': 'Pikachu', 'french': 'Pikac...",['Electric'],"{'HP': 35, 'Attack': 55, 'Defense': 40, 'Sp. Attack': 50,...",Mouse Pokémon,"{'next': [['26', 'use Thunder Stone']], 'prev': ['172', '...","{'ability': [['Static', 'false'], ['Lightning Rod', 'true...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,24,"While sleeping, it generates electricity in the sacs in i...","<np.ndarrayshape=(768,)dtype=float64>",0.474437
328,"{'chinese': '大颚蚁', 'english': 'Trapinch', 'french': 'Krak...",['Ground'],"{'HP': 45, 'Attack': 100, 'Defense': 45, 'Sp. Attack': 45...",Ant Pit Pokémon,"{'next': [['329', 'Level 35']], 'prev': None}","{'ability': [['Hyper Cutter', 'false'], ['Arena Trap', 'f...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,327,"It builds nests that double as traps, lying in wait for i...","<np.ndarrayshape=(768,)dtype=float64>",0.450725
722,"{'chinese': '木木枭', 'english': 'Rowlet', 'french': 'Brindi...","['Grass', 'Flying']","{'HP': 68, 'Attack': 55, 'Defense': 55, 'Sp. Attack': 50,...",Grass Quill Pokémon,"{'next': [['723', 'Level 17']], 'prev': None}","{'ability': [['Overgrow', 'false'], ['Long Reach', 'true'...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,721,"It feels relaxed in tight, dark places and has been known...","<np.ndarrayshape=(768,)dtype=float64>",0.445208
554,"{'chinese': '火红不倒翁', 'english': 'Darumaka', 'french': 'Da...",['Fire'],"{'HP': 70, 'Attack': 90, 'Defense': 45, 'Sp. Attack': 15,...",Zen Charm Pokémon,"{'next': [['555', 'use Ice Stone']], 'prev': None}","{'ability': [['Hustle', 'false'], ['Inner Focus', 'true']...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,553,"When it sleeps, it pulls its limbs into its body and its ...","<np.ndarrayshape=(768,)dtype=float64>",0.430829


#### Batch querying -- return a list of collections for each query

In [20]:
sleepy_pokemon_batch = collection.batch_query(
    ["embeddings_description", "embeddings_description", "embeddings_description"],
    queries=["Loves to sleep", "Fire Pokemon", "Lives in water"],
    k=10,
    sort=True
)

In [21]:
sleepy_pokemon_batch[2].show(5)

idInt64,"nameStruct[chinese: Utf8, english: Utf8, french: Utf8, japanese: Utf8]",typeList[Utf8],basePython,speciesUtf8,"evolutionStruct[next: List[List[Utf8]], prev: List[Utf8]]","profileStruct[ability: List[List[Utf8]], egg: List[Utf8], gender: Utf8, height: Utf8, weight: Utf8]","imageStruct[hires: Utf8, sprite: Utf8, thumbnail: Utf8]",vexpresso_indexInt64,descriptionUtf8,embeddings_descriptionPython,embeddings_description_scoreFloat64
536,"{'chinese': '蓝蟾蜍', 'english': 'Palpitoad', 'french': 'Bat...","['Water', 'Ground']","{'HP': 75, 'Attack': 65, 'Defense': 55, 'Sp. Attack': 65,...",Vibration Pokémon,"{'next': [['537', 'Level 36']], 'prev': ['535', 'Level 25']}","{'ability': [['Swift Swim', 'false'], ['Hydration', 'fals...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,535,"It lives in the water and on land. It uses its long, stic...","<np.ndarrayshape=(768,)dtype=float64>",0.709986
751,"{'chinese': '滴蛛', 'english': 'Dewpider', 'french': 'Araqu...","['Water', 'Bug']","{'HP': 38, 'Attack': 40, 'Defense': 52, 'Sp. Attack': 40,...",Water Bubble Pokémon,"{'next': [['752', 'Level 22']], 'prev': None}","{'ability': [['Water Bubble', 'false'], ['Water Absorb', ...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,750,"It can only breathe oxygen that has dissolved in water, s...","<np.ndarrayshape=(768,)dtype=float64>",0.654163
171,"{'chinese': '电灯怪', 'english': 'Lanturn', 'french': 'Lantu...","['Water', 'Electric']","{'HP': 125, 'Attack': 58, 'Defense': 58, 'Sp. Attack': 76...",Light Pokémon,"{'next': None, 'prev': ['170', 'Level 27']}","{'ability': [['Volt Absorb', 'false'], ['Illuminate', 'fa...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,170,It lives far down in the depths of the ocean. It blinds i...,"<np.ndarrayshape=(768,)dtype=float64>",0.64507
457,"{'chinese': '霓虹鱼', 'english': 'Lumineon', 'french': 'Lumi...",['Water'],"{'HP': 69, 'Attack': 69, 'Defense': 76, 'Sp. Attack': 69,...",Neon Pokémon,"{'next': None, 'prev': ['456', 'Level 31']}","{'ability': [['Swift Swim', 'false'], ['Storm Drain', 'fa...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,456,They traverse the deep waters as if crawling over the sea...,"<np.ndarrayshape=(768,)dtype=float64>",0.623969
121,"{'chinese': '宝石海星', 'english': 'Starmie', 'french': 'Star...","['Water', 'Psychic']","{'HP': 60, 'Attack': 75, 'Defense': 85, 'Sp. Attack': 100...",Mysterious Pokémon,"{'next': None, 'prev': ['120', 'use Water Stone']}","{'ability': [['Illuminate', 'false'], ['Natural Cure', 'f...",{'hires': 'https://raw.githubusercontent.com/Purukitto/po...,120,It rotates its geometrically shaped body to swim through ...,"<np.ndarrayshape=(768,)dtype=float64>",0.615116


### This allows for query chaining

In [ ]:
sleepy_water_pokemon = collection.query("embeddings_description", query=["Loves to sleep"], k=10) \
                                 .query("embeddings_description", query=["Lives in water"], k=5) \
                                 .execute()

#### Show daft DataFrame

In [ ]:
sleepy_water_pokemon.show(5)

#### Show pandas

In [ ]:
sleepy_water_pokemon.to_pandas()

## Querying the collection's embeddings, eager execution
### This is useful if you don't need to chain calls together and just want to get the results back immediately

In [ ]:
sleepy_pokemon = collection.query("embeddings_description", query=["Loves to sleep"], k=10, lazy=False)
sleepy_pokemon.show(5)

## Performing filters

### With vexpresso we can leverage duckdb for powerful queries

#### In order to filter on metadata, you must supply a filter dictionary to the filter method. The dictionary must have the following structure:

```python
{
    <field>: {
        <filter_method>: <value>
    },
    <field>: {
        <filter_method>: <value>
    },
}

```

#### Query to only get Pokemon with the name "Snorlax"

In [ ]:
filter_condition = {
    "name.english": {
        "eq":"Snorlax"
    },
}

sleepy_pokemon.filter(filter_condition).to_pandas()

#### Query to only get Pokemon with the name "Snorlax" or "Pikachu"

In [ ]:
filter_condition = {
    "name.english": [
        {
            "eq":"Snorlax"
        },
        {
            "eq":"Pikachu"
        },
    ],
}
sleepy_pokemon.filter(filter_condition).to_pandas()

#### Query to only get Pokemon with the name "Snorlax" or "Pikachu" using a list

In [ ]:
filter_condition = {
    "name.english": {
        "isin":["Snorlax", "Pikachu"]
    }
}
sleepy_pokemon.filter(filter_condition).to_pandas()

#### Query to only get Pokemon with type Normal and attack > 100

In [ ]:
filter_condition = {
    "type": {
        "contains":"Normal"
    },
    "base.Attack": {
        "gt":100
    }
}
sleepy_pokemon.filter(filter_condition).to_pandas()

## Query + Filter chaining

#### For example, let's filter the above collection for Pokemon with HP >= 35 and Defense > 45

In [ ]:
filter_conditions = {
    "base.HP": {"gte":35},
    "base.Defense": {"gt":45}
}
sleepy_pokemon = collection.query("embeddings_description", query=["Loves to sleep"], k=10).filter(filter_conditions=filter_conditions).execute()
sleepy_pokemon.show(5)

## Grab fields from metadata

In [ ]:
names, types, descriptions = sleepy_pokemon.select("name.english", "type", "description").to_list()
for n, t, d in zip(names, types, descriptions):
    print(f"{n}: {t} -- {d}")

## Save collection

In [ ]:
sleepy_pokemon.save("./saved_filtered_pokemon")

## Load collection

In [ ]:
loaded_collection = vexpresso.create(
    directory_or_repo_id = "saved_filtered_pokemon",
    embedding_functions = {"embeddings_description":hf_embedding_function}
)

In [ ]:
loaded_collection.show(5)

## Uploading to HFhub

In [ ]:
# Automatically gets token from env variable
# HUGGINGFACEHUB_API_TOKEN = ...
# or you can pass in token directly via collection.save(token=...)
# repo_id = {HF_USERNAME}/{REPO_NAME}
repo_id = "shyamsn97/pokemon_vexpresso_test"
loaded_collection.save(repo_id, to_hub=True, private=True)

In [ ]:
# or upload with username + repo name
loaded_collection.save(hf_username = "shyamsn97", repo_name = "pokemon_vexpresso_test", to_hub=True)

## Load from HfHub

In [ ]:
# Automatically gets token from env variable
# HUGGINGFACEHUB_API_TOKEN = ...
# or you can pass in token directly via Collection.load(token=...)
# repo_id = {HF_USERNAME}/{REPO_NAME}
repo_id = "shyamsn97/pokemon_vexpresso_test"

collection = vexpresso.create(
    directory_or_repo_id = repo_id,
    embedding_functions = {"embeddings_description":hf_embedding_function}
)
collection.to_pandas()

In [ ]:
# or load with username + repo name
collection = vexpresso.create(
    hf_username = "shyamsn97",
    repo_name = "pokemon_vexpresso_test",
    embedding_functions = {"embeddings_description":hf_embedding_function}
)
collection.to_pandas()